# Lab 3: Sampling and visualizations

Welcome to lab 3!  We're going to take a breather from learning Python and focus on some basic things we can do with tables.  In this lab:
1. We'll see several ways to make visualizations.
2. We'll use tables to simulate random sampling.
3. We'll use histograms to visualize the results of sampling.

The cell below imports the packages we'll use so you don't have to worry about it later.  It also takes a few lines to set up visualizations in Jupyter notebooks.  Run it before you continue.

In [83]:
# Run this cell, but please don't change it.

# These lines import the NumPy, datascience, and math modules.
import numpy as np
# This way of importing the datascience module lets you write "Table" instead
# of "datascience.Table".  The "*" means "import everything in the module."
from datascience import *
import math

# These lines set up visualizations.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

# These lines load the tests.
from client.api.assignment import load_assignment 
lab03 = load_assignment('longlab03.ok')

# 1. Visualizations
In the previous lab, we worked with data on world population.  Here we've loaded it again into a table, now with US population for each year.

In [12]:
population = Table.read_table("population.csv")
population

## 1.1. Line plots
Suppose we want to see how world population has changed over time.  We could read the data in the table (calling `population.show()` will show the entire table), but that's a pretty bad way of looking at numbers.  Instead, let's make a line plot:

In [13]:
population.plot("Year", "World Population")

The first argument to `plot` is the name of the column with the x-axis data, and the second is the name of the column with the y-axis data.

We can also plot US and world population together.  When we omit the second argument to `plot`, it makes a separate curve with y-axis data from each column other than "Year".  Since the two other columns in our table are "World Population" and "US Population", this plots both the world and US population on the same graph.

In [14]:
population.plot("Year")

Unfortunately, it's hard to compare them on this scale.  This is a common problem with visualizations.  We could try comparing their orders of magnitude instead.

To do that, we'll make a new table with the orders of magnitude.  We'll use `np.log10` on the US and world population columns (which are just arrays), like we did in the previous lab.

In [16]:
# It's possible to substantially shorten the code we've written
# here, and later we will use those shortened forms.  For now we've
# written it in a fairly verbose way, for clarity.

# Start with an empty table, produced by calling Table().
# The idea is that we'll grow it with each column we want it to have.
empty = Table()

# Extend that table with the column of years.
with_years = empty.with_column("Year", population.column("Year"))

# Compute the log US populations.  This is an array of
# numbers, each the logarithm of the US population in
# a different year.
log_us_population = np.log10(population.column("US Population"))

# Extend our table with a column of log US populations.
with_us = with_years.with_column("US Population (log)", log_us_population)

# Compute the log world populations.
log_world_population = np.log10(population.column("World Population"))

# Extend our table one last time with a column of log
# world populations.
magnitudes = with_us.with_column("World Population (log)", log_world_population)

# Now we can make our plot.
magnitudes.plot("Year")

The US line is slightly less steep than the world line, so the US population has been growing a bit more slowly than the world population.

## 1.2. Scatter plots
Now let's look at the IMDb movie dataset.  This dataset includes a few hundred highly-rated movies.

In [18]:
imdb = Table.read_table("imdb_ratings.csv")
imdb

Suppose we want to see if there's a relationship between the number of votes movies receive and their age, at least among these high-quality movies.  We might hypothesize that older movies receive fewer votes, for example.

In [21]:
imdb.scatter("Year", "Votes")

It looks like older movies indeed tend to receive fewer votes.

**Question 1.2.1.** Make a scatter plot to compare the year a movie came out and the *decade* in which it came out.  Before you draw the plot, see if you can predict what the plot will look like, assuming we've computed the decades correctly.

In [22]:
...

## 1.3. Histograms
From the first scatter plot we drew, we might get some sense of how many movies from each time period are in our dataset.  There are more dots in the later periods than the earlier ones.  But a *histogram* is a much more helpful for this kind of question:

In [30]:
imdb.hist("Year", unit="year")

The first argument to `hist` is the name of the column containing the data we want to display.  The `unit="year"` part tells the plot to say "Percent per **year**" on the y-axis.  It's called a *named argument* because we write the name of the argument, `unit` in addition to the argument itself, which is `"year"`.  Some functions work this way because it's more clear what the argument does.

`hist` puts the data into "bins," and each bar represents the data in one bin.  For example, the third bar from the left bar covers the years from 1940 to 1949.  It has height around 0.6 and width 9 (1940-1949), so its area is around 5.4.  That means 5.4% of the movies are from 1940 to 1949.

`hist` chose bins of width 9 rather than 10, which is a little weird.  We can override the default by passing an array of numbers to `hist`, like this:

In [31]:
every_decade = np.arange(1920, 2030, 10)
imdb.hist("Year", unit="year", bins=every_decade)

Bins are a little confusing, and the defaults are often okay, so we won't dwell on them too much.

**Question 1.3.1.** Make a histogram of the movie ratings.

In [33]:
# This is a good array to use for bins.
every_tenth = np.arange(8.0, 9.4, .1)
...

# 2. Sampling
Let's go back to the flight delay data from today's lecture.  Try running the next cell to look at the data, then adding code to the cell to make a histogram of the delays.

In [98]:
united = Table.read_table("united.csv")

# Use these bins for your histogram:
delay_bins = np.arange(-10, 310, 10)
# Fill in this line to make a histogram of the delays.
# Use units called "minute".
...

united

Our dataset has information about 13732 flights.  Often we don't have access to all the data we want; perhaps we only have a *random sample* of them.

For example, maybe the airlines don't want to reveal all their data.  Instead, they put the flight numbers on 13732 cards, shuffle the cards, pick 50 of them, and show us only the flights on those 50 cards.  We call the 13732 flights the *population* and the 50 flights a *uniform random sample without replacement* from them.

We can simulate this using the method `sample`.

In [110]:
sample_size = 50
united_sample = united.sample(sample_size)
united_sample

`united_sample` is a table like `united`, with the same columns.  But only 50 of the original thousands of rows remain.  (The table `united` is unaffected by this; it still contains all the data.  You can verify that by writing `united` in a code cell somewhere.)

It is an amazing fact that we can sometimes make strong -- though not airtight -- inferences about the population of flights, using only the sample.  Let's see if that's true here.

Suppose we want to know what the flight delays look like.  If we plot a histogram of the flight delays in the sample, here's what we get:

In [111]:
united_sample.hist("Delay", unit="minute", bins=delay_bins)
# Add a title to our plot.  Note that "\n" means "make a new line."
_ = plt.title("Flight delays in a random\nsample of " + str(sample_size) + " flights")

You probably see a histogram that vaguely resembles the population histogram, but it's not a perfect match.  We could answer some coarse questions accurately (like "are most of the delays less than 50 minutes?"), but individual bin sizes are likely to be a little off.

If we use a larger sample, we will usually get something closer.

In [112]:
larger_size = 1000
larger_united_sample = united.sample(larger_size)
larger_united_sample.hist("Delay", unit="minute", bins=delay_bins)
_ = plt.title("Flight delays in a random\nsample of " + str(larger_size) + " flights")

Note that this is not a guarantee.  We might, for example, get unlucky enough to see a sample of the 1000 smallest delay times.  Because it's extremely unlikely, we'll pretend it happened rather than waiting for it to happen:

In [115]:
unlucky_united_sample = united.sort("Delay").take(np.arange(larger_size))
unlucky_united_sample.hist("Delay", unit="minute", bins=delay_bins)
_ = plt.title("Flight delays among the " + str(larger_size) + "\nshortest-delay flights")

If we assumed this was similar to the population, we'd think there were no flights with delays in `united`!  That would be an incorrect conclusion, of course.

When we take a random sample of 1000 flights, all possible samples are equally likely, so we're just as likely to see this unlucky sample as `larger_united_sample`.  But there are many, many possible samples:

In [114]:
num_possible_samples = math.factorial(united.num_rows) // (math.factorial(larger_size)*math.factorial(united.num_rows - larger_size))
num_possible_samples

Most of those possible samples of 1000 flights produce histograms that are roughly similar to the histogram of flight delays in `united`.  So if we only had access to a large random sample, not the `united` table itself, we could probably make reasonable conclusions from that histogram, subject to a little random error.

It wasn't clear a priori that the sample of 50 would work poorly and the sample of 1000 would work pretty well.  And if we wanted very precise information about the flight delays in `united`, even the sample of 1000 might not suffice.  Much of the statistical content of this course is about characterizing, more precisely, the error that arises when we make conclusions from random samples.